In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 457 ms (started: 2021-08-28 21:31:30 +08:00)


In [3]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 #1 Aug 29, 2021'

#git push origin master
git push

origin	git@github.com:ustchope/pytorch_lightning_study.git (fetch)
origin	git@github.com:ustchope/pytorch_lightning_study.git (push)
[main cd05707] 更新 #4 Aug 28, 2021
 2 files changed, 484 insertions(+), 1 deletion(-)
 create mode 100644 LIGHTNINGMODULE.ipynb


To git@github.com:ustchope/pytorch_lightning_study.git
   0c6ecfd..cd05707  main -> main


time: 5.65 s (started: 2021-08-28 20:46:50 +08:00)


In [2]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl

time: 3.98 s (started: 2021-08-28 21:31:34 +08:00)


# LightningModule

LightningModule 将您的 PyTorch 代码组织成 5 个部分
* 计算（`Init`）。
* 训练循环（`training_step`）
* 验证循环（`validation_step`）
* 测试循环（`test_step`）
* 优化器（`configure_optimizers`）

**注意一些事情**

1. 这是相同的代码。
2.  PyTorch 代码不是抽象的——只是有组织的。
3.  不在 LightningModule 中的所有其他代码已由trainer为您自动化。

```
net = Net()
trainer = Trainer()
trainer.fit(net)

```
4. 没有 `.cuda()` 或 `.to()` 调用……Lightning 为你做这些。

```
# don't do in lightning
x = torch.Tensor(2, 3)
x = x.cuda()
x = x.to(device)

# do this instead
x = x  # leave it alone!

# or to init a new tensor
new_x = torch.Tensor(2, 3)
new_x = new_x.type_as(x)
```

5. 默认情况下，Lightning 会为您处理分布式采样器。  

```
# Don't do in Lightning...
data = MNIST(...)
sampler = DistributedSampler(data)
DataLoader(data, sampler=sampler)

# do this instead
data = MNIST(...)
DataLoader(data)
```

6. LightningModule 是一个 torch.nn.Module，但具有附加功能。 这样使用它！

```
net = Net.load_from_checkpoint(PATH)
net.freeze()
out = net(x)
```

因此，要使用 Lightning，你只需要组织你的代码，大约需要 30 分钟，（让我们现实点，你可能应该做任何事情）。

# 最小示例

以下是唯一需要的方法。

In [4]:
import pytorch_lightning as pl


class LitModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(28 * 28, 10)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

time: 860 µs (started: 2021-08-28 21:32:22 +08:00)


您可以通过以下方式进行训练：

In [8]:
train_loader = DataLoader(MNIST(os.getcwd(), 
                                download=True, 
                                transform=transforms.ToTensor()), 
                          num_workers = 8)
trainer = pl.Trainer(gpus=1)
model = LitModel()

trainer.fit(model, train_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 7.9 K 
--------------------------------
7.9 K     Trainable params
0         Non-trainable params
7.9 K     Total params
0.031     Total estimated model params size (MB)


Training: -1it [00:00, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



time: 4h 42min 33s (started: 2021-08-28 22:56:21 +08:00)


LightningModule 有很多方便的方法，但你需要了解的核心方法是：

|名称|描述|
|----|----|
|init|在此处定义计算|
|forward|仅用于推理（与 training_step 分开）|
|training_step|完整的训练循环|
|validation_step|完整的验证循环|
|test_step|完整的测试循环|
|configure_optimizers|定义优化器和 LR 调度器|

# 训练

## 训练循环

要添加训练循环，请使用 `training_step` 方法

In [3]:
class LitClassifier(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        return loss

time: 743 µs (started: 2021-08-28 21:32:12 +08:00)


在幕后，Lightning 执行以下操作（伪代码）：

In [ ]:
# put model in train mode
model.train()
torch.set_grad_enabled(True)

losses = []
for batch in train_dataloader:
    # forward
    loss = training_step(batch)
    losses.append(loss.detach())

    # clear gradients
    optimizer.zero_grad()

    # backward
    loss.backward()

    # update parameters
    optimizer.step()

## 训练Epoch级别的指标

如果要计算Epoch级指标并记录它们，请使用 `.log` 方法

In [ ]:
def training_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self.model(x)
    loss = F.cross_entropy(y_hat, y)

    # 记录每个training_step 的指标，
    # 和整个Epoch的平均值，到进度条和记录器
    self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    return loss

.log 对象会自动减少整个 epoch 中请求的指标。 这是它在幕后所做的伪代码：

In [ ]:
outs = []
for batch in train_dataloader:
    # forward
    out = training_step(val_batch)
    outs.append(out)

    # clear gradients
    optimizer.zero_grad()

    # backward
    loss.backward()

    # update parameters
    optimizer.step()

epoch_metric = torch.mean(torch.stack([x["train_loss"] for x in outs]))

## 训练Epoch级操作

如果您需要对每个 training_step 的所有输出执行某些操作，请自己覆盖 training_epoch_end。

In [ ]:
def training_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self.model(x)
    loss = F.cross_entropy(y_hat, y)
    preds = ...
    return {"loss": loss, "other_stuff": preds}


def training_epoch_end(self, training_step_outputs):
    for pred in training_step_outputs:
        ...

匹配的伪代码是：

In [ ]:
outs = []
for batch in train_dataloader:
    # forward
    out = training_step(val_batch)
    outs.append(out)

    # clear gradients
    optimizer.zero_grad()

    # backward
    loss.backward()

    # update parameters
    optimizer.step()

training_epoch_end(outs)

## 使用 DataParallel 进行训练

当使用加速器将每个批次的数据拆分到 GPU 上时，有时您可能需要在主 GPU 上聚合它们以进行处理（dp 或 ddp2）。

在这种情况下，实现 training_step_end 方法

In [ ]:
def training_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self.model(x)
    loss = F.cross_entropy(y_hat, y)
    pred = ...
    return {"loss": loss, "pred": pred}


def training_step_end(self, batch_parts):
    # 每个 GPU 的预测
    predictions = batch_parts["pred"]
    # 每个 GPU 的损失
    losses = batch_parts["loss"]

    gpu_0_prediction = predictions[0]
    gpu_1_prediction = predictions[1]

    # 对两个输出做一些事情
    return (losses[0] + losses[1]) / 2


def training_epoch_end(self, training_step_outputs):
    for out in training_step_outputs:
        ...

PL在幕后所做的完整伪代码是：

In [ ]:
outs = []
for train_batch in train_dataloader:
    batches = split_batch(train_batch)
    dp_outs = []
    for sub_batch in batches:
        # 1
        dp_out = training_step(sub_batch)
        dp_outs.append(dp_out)

    # 2
    out = training_step_end(dp_outs)
    outs.append(out)

# do something with the outputs for all batches
# 3
training_epoch_end(outs)

# 验证循环

要添加验证循环，请覆盖 LightningModule 的 validation_step 方法：

In [ ]:
class LitModel(pl.LightningModule):
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("val_loss", loss)

在幕后，Lightning 执行以下操作：

In [ ]:
# ...
for batch in train_dataloader:
    loss = model.training_step()
    loss.backward()
    # ...

    if validate_at_some_point:
        # disable grads + batchnorm + dropout
        torch.set_grad_enabled(False)
        model.eval()

        # ----------------- VAL LOOP ---------------
        for val_batch in model.val_dataloader:
            val_out = model.validation_step(val_batch)
        # ----------------- VAL LOOP ---------------

        # enable grads + batchnorm + dropout
        torch.set_grad_enabled(True)
        model.train()

## 验证Epoch级别的指标

如果您需要对每个validation_step 的所有输出执行某些操作，请覆盖validation_epoch_end。

In [ ]:
def validation_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self.model(x)
    loss = F.cross_entropy(y_hat, y)
    pred = ...
    return pred


def validation_epoch_end(self, validation_step_outputs):
    for pred in validation_step_outputs:
        ...

## 使用 DataParallel 进行验证

当使用加速器将每个批次的数据拆分到 GPU 上时，有时您可能需要在主 GPU 上聚合它们以进行处理（dp 或 ddp2）。

在这种情况下，实现validation_step_end 方法

In [ ]:
def validation_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self.model(x)
    loss = F.cross_entropy(y_hat, y)
    pred = ...
    return {"loss": loss, "pred": pred}


def validation_step_end(self, batch_parts):
    # predictions from each GPU
    predictions = batch_parts["pred"]
    # losses from each GPU
    losses = batch_parts["loss"]

    gpu_0_prediction = predictions[0]
    gpu_1_prediction = predictions[1]

    # do something with both outputs
    return (losses[0] + losses[1]) / 2


def validation_epoch_end(self, validation_step_outputs):
    for out in validation_step_outputs:
        ...

PL在幕后所做的完整伪代码是：

In [ ]:
outs = []
for batch in dataloader:
    batches = split_batch(batch)
    dp_outs = []
    for sub_batch in batches:
        # 1
        dp_out = validation_step(sub_batch)
        dp_outs.append(dp_out)

    # 2
    out = validation_step_end(dp_outs)
    outs.append(out)

# do something with the outputs for all batches
# 3
validation_epoch_end(outs)

# 测试循环

添加测试循环的过程与添加验证循环的过程相同。 详情请参阅上一节。

唯一的区别是只有在使用 .test() 时才会调用测试循环：

In [ ]:
model = Model()
trainer = Trainer()
trainer.fit()

# automatically loads the best weights for you
trainer.test(model)

有两种方法可以调用 test()：

In [ ]:
# 训练后调用
trainer = Trainer()
trainer.fit(model)

# 自动加载最佳权重
trainer.test(dataloaders=test_dataloader)

# 或使用预训练模型调用
model = MyLightningModule.load_from_checkpoint(PATH)
trainer = Trainer()
trainer.test(model, dataloaders=test_dataloader)

# 推理

对于研究，LightningModules 是作为系统的最好结构。

In [ ]:
import pytorch_lightning as pl
import torch
from torch import nn


class Autoencoder(pl.LightningModule):
    def __init__(self, latent_dim=2):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(28 * 28, 256), nn.ReLU(), nn.Linear(256, latent_dim))
        self.decoder = nn.Sequential(nn.Linear(latent_dim, 256), nn.ReLU(), nn.Linear(256, 28 * 28))

    def training_step(self, batch, batch_idx):
        x, _ = batch

        # encode
        x = x.view(x.size(0), -1)
        z = self.encoder(x)

        # decode
        recons = self.decoder(z)

        # reconstruction
        reconstruction_loss = nn.functional.mse_loss(recons, x)
        return reconstruction_loss

    def validation_step(self, batch, batch_idx):
        x, _ = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        recons = self.decoder(z)
        reconstruction_loss = nn.functional.mse_loss(recons, x)
        self.log("val_reconstruction", reconstruction_loss)

    def predict_step(self, batch, batch_idx, dataloader_idx):
        x, _ = batch

        # encode
        # 对于预测，我们可以根据需要返回嵌入或重建或两者。
        x = x.view(x.size(0), -1)
        return self.encoder(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.0002)

可以这样训练：

In [ ]:
autoencoder = Autoencoder()
trainer = pl.Trainer(gpus=1)
trainer.fit(autoencoder, train_dataloader, val_dataloader)

这个简单的模型生成的例子看起来像这样（编码器和解码器太弱了）

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtwwudesc9j614q06sjrp02.jpg)

以上方法是闪lightning接口的一部分：
* training_step
* validation_step
* test_step
* predict_step
* configure_optimizers

请注意，在这种情况下，train 循环和 val 循环完全相同。 我们当然可以重用这段代码。

In [ ]:
class Autoencoder(pl.LightningModule):
    def __init__(self, latent_dim=2):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(28 * 28, 256), nn.ReLU(), nn.Linear(256, latent_dim))
        self.decoder = nn.Sequential(nn.Linear(latent_dim, 256), nn.ReLU(), nn.Linear(256, 28 * 28))

    def training_step(self, batch, batch_idx):
        loss = self.shared_step(batch)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.shared_step(batch)
        self.log("val_loss", loss)

    def shared_step(self, batch):
        x, _ = batch

        # encode
        x = x.view(x.size(0), -1)
        z = self.encoder(x)

        # decode
        recons = self.decoder(z)

        # loss
        return nn.functional.mse_loss(recons, x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.0002)

我们创建了一个名为 shared_step 的新方法，所有循环都可以使用它。 此方法名称是任意的，不保留。

## 研究中的推理

在我们想要对系统进行推理的情况下，我们可以向 LightningModule 添加一个 forward 方法。

> 使用 forward 时，您负责调用 eval() 并使用 no_grad() 上下文管理器。

In [ ]:
class Autoencoder(pl.LightningModule):
    def forward(self, x):
        return self.decoder(x)


model = Autoencoder()
model.eval()
with torch.no_grad():
    reconstruction = model(embedding)

添加前向的好处在于，在复杂系统中，您可以执行更复杂的推理过程，例如文本生成：

In [ ]:
class Seq2Seq(pl.LightningModule):
    def forward(self, x):
        embeddings = self(x)
        hidden_states = self.encoder(embeddings)
        for h in hidden_states:
            # decode
            ...
        return decoded

在您想要扩展推理的情况下，您应该使用 predict_step()。

In [ ]:
class Autoencoder(pl.LightningModule):
    def forward(self, x):
        return self.decoder(x)

    def predict_step(self, batch, batch_idx, dataloader_idx=None):
        # 这里调用 forward
        return self(batch)


data_module = ...
model = Autoencoder()
trainer = Trainer(gpus=2)
trainer.predict(model, data_module)

## 生产中的推理

对于生产等情况，您可能希望在 LightningModule 中迭代不同的模型。

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.metrics import functional as FM


class ClassificationTask(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch, batch_idx)
        metrics = {"val_acc": acc, "val_loss": loss}
        self.log_dict(metrics)
        return metrics

    def test_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch, batch_idx)
        metrics = {"test_acc": acc, "test_loss": loss}
        self.log_dict(metrics)
        return metrics

    def _shared_eval_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        acc = FM.accuracy(y_hat, y)
        return loss, acc

    def predict_step(self, batch, batch_idx, dataloader_idx):
        x, y = batch
        y_hat = self.model(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=0.02)

然后传入任何适合此任务的任意模型

In [ ]:
for model in [resnet50(), vgg16(), BidirectionalRNN()]:
    task = ClassificationTask(model)

    trainer = Trainer(gpus=2)
    trainer.fit(task, train_dataloader, val_dataloader)

任务可以是任意复杂的，例如实施 GAN 训练、自我监督甚至 RL。

In [ ]:
class GANTask(pl.LightningModule):
    def __init__(self, generator, discriminator):
        super().__init__()
        self.generator = generator
        self.discriminator = discriminator

    ...

像这样使用时，模型可以与任务分离，从而在生产中使用，而无需将其保存在 LightningModule 中。
* 您可以导出到onnx。
* 或者使用 Jit 进行跟踪。
* 或在 python 运行时中运行。

In [ ]:
task = ClassificationTask(model)

trainer = Trainer(gpus=2)
trainer.fit(task, train_dataloader, val_dataloader)

# 训练后使用模型或加载权重并放入生产系统
model.eval()
y_hat = model(x)

# LightningModule API

## 方法

### configure_callbacks
```
LightningModule.configure_callbacks()
```

配置特定于模型的回调。 当模型被附加时，例如，当 `.fit()` 或 `.test()` 被调用时，此处返回的列表将与传递给 Trainer 的 callbacks 参数的回调列表合并。 如果此处返回的回调与 Trainer 的回调列表中已存在的一个或多个回调具有相同的类型，则它将优先并替换它们。 此外，Lightning 将确保 `ModelCheckpoint` 回调最后运行。

返回：
一个回调列表，它将扩展 `Trainer` 中的回调列表。

例子：

In [ ]:
def configure_callbacks(self):
    early_stop = EarlyStopping(monitor"val_acc", mode="max")
    checkpoint = ModelCheckpoint(monitor="val_loss")
    return [early_stop, checkpoint]

> 某些回调方法如 on_init_start() 永远不会在此处返回的新回调上调用。

### configure_optimizers
```
LightningModule.configure_optimizers()
```

选择在优化中使用的优化器和学习率调度器。 通常你需要一个。 但是在 GAN 或类似的情况下，您可能有多个。

返回
这6个选项中的任何一个。
* 单个优化器。
* 优化器列表或元组。
* 两个列表 - 第一个列表有多个优化器，第二个列表有多个 LR 调度程序（或多个 `lr_dict`）。
* 字典，带有“优化器”键和（可选）“`lr_scheduler`”键，其值为单个 LR 调度器或 `lr_dict`。
* 如上所述的字典元组，带有可选的“频率”键。
* 无 - Fit 将在没有任何优化器的情况下运行。

`lr_dict` 是一个包含调度程序及其相关配置的字典。 默认配置如下所示。

In [ ]:
lr_dict = {
    # REQUIRED: 调度器实例
    "scheduler": lr_scheduler,
    # 调度器步长的单位，也可以是“step”。 
    # 'epoch' 在 epoch 结束时更新调度程序，而 'step' 在优化程序更新后更新它。
    "interval": "epoch",
    # 在调用 `scheduler.step()` 之间应该传递多少个 epochs/steps。 
    # 1 对应于在每个 epoch/step 之后更新学习率。
    "frequency": 1,
    # 监控调度程序的指标，如“ReduceLROnPlateau”
    "monitor": "val_loss",
    # 如果设置为 `True`，将强制指定的值在更新调度程序时可用，因此如果未找到则停止训练。 
    # 如果设置为`False`，它只会产生一个警告
    "strict": True,
    # 如果使用`LearningRateMonitor`回调来监控学习率进度，
    # 这个关键字可以用来指定一个自定义的日志名称
    "name": None,
}

当存在其中 `.step()` 方法以值为条件的调度程序时，例如 `torch.optim.lr_scheduler.ReduceLROnPlateau` 调度程序，Lightning 要求 `lr_dict` 包含关键字“`monitor`”设置为调度程序应的指标名称 有条件。`

In [ ]:
# ReduceLROnPlateau 调度器需要一个监视器
def configure_optimizers(self):
    optimizer = Adam(...)
    return {
        "optimizer": optimizer,
        "lr_scheduler": {
            "scheduler": ReduceLROnPlateau(optimizer, ...),
            "monitor": "metric_to_track",
        },
    }


# 在两个优化器的情况下，只有一个使用 ReduceLROnPlateau 调度器
def configure_optimizers(self):
    optimizer1 = Adam(...)
    optimizer2 = SGD(...)
    scheduler1 = ReduceLROnPlateau(optimizer1, ...)
    scheduler2 = LambdaLR(optimizer2, ...)
    return (
        {
            "optimizer": optimizer1,
            "lr_scheduler": {
                "scheduler": scheduler1,
                "monitor": "metric_to_track",
            },
        },
        {"optimizer": optimizer2, "lr_scheduler": scheduler2},
    )

可以通过在 LightningModule 中使用 `self.log('metric_to_track', metric_val)` 简单地记录来使指标可用于监控。

---

dict 中指定的`frequency`值与优化器键一起是一个 int，对应于使用特定优化器优化的连续批次的数量。 它应该被赋予一个或所有的优化器。 在列表中传递多个优化器和在字典中以 1 的频率传递多个优化器之间存在差异：
* 在前一种情况下，所有优化器将在每个优化步骤中对给定的批次进行操作。
* 在后者中，每一步只有一个优化器会对给定的批次进行操作。

这与上面提到的 lr_dict 中指定的频率值不同。

```
def configure_optimizers(self):
    optimizer_one = torch.optim.SGD(self.model.parameters(), lr=0.01)
    optimizer_two = torch.optim.SGD(self.model.parameters(), lr=0.01)
    return [
        {"optimizer": optimizer_one, "frequency": 5},
        {"optimizer": optimizer_two, "frequency": 10},
    ]
```

在此示例中，第一个优化器将用于前 5 个步骤，第二个优化器用于接下来的 10 个步骤，并且该循环将继续。 如果使用上述字典中的 `lr_scheduler` 键为优化器指定了 LR 调度器，则调度器将仅在使用其优化器时更新。

---

例子：

In [ ]:
# 在大多数情况下。 没有学习率调度程序
def configure_optimizers(self):
    return Adam(self.parameters(), lr=1e-3)

# 多个优化器案例（例如：GAN）
def configure_optimizers(self):
    gen_opt = Adam(self.model_gen.parameters(), lr=0.01)
    dis_opt = Adam(self.model_dis.parameters(), lr=0.02)
    return gen_opt, dis_opt

# 学习率调度器示例
def configure_optimizers(self):
    gen_opt = Adam(self.model_gen.parameters(), lr=0.01)
    dis_opt = Adam(self.model_dis.parameters(), lr=0.02)
    dis_sch = CosineAnnealing(dis_opt, T_max=10)
    return [gen_opt, dis_opt], [dis_sch]

# 基于步骤的学习率调度器的示例
# 每个优化器都有自己的调度器
def configure_optimizers(self):
    gen_opt = Adam(self.model_gen.parameters(), lr=0.01)
    dis_opt = Adam(self.model_dis.parameters(), lr=0.02)
    gen_sch = {
        'scheduler': ExponentialLR(gen_opt, 0.99),
        'interval': 'step'  # called after each training step
    }
    dis_sch = CosineAnnealing(dis_opt, T_max=10) # called every epoch
    return [gen_opt, dis_opt], [gen_sch, dis_sch]

# 优化器频率示例
# 参见“改进的 Wasserstein GAN 训练”中的训练程序，算法 1
# https://arxiv.org/abs/1704.00028
def configure_optimizers(self):
    gen_opt = Adam(self.model_gen.parameters(), lr=0.01)
    dis_opt = Adam(self.model_dis.parameters(), lr=0.02)
    n_critic = 5
    return (
        {'optimizer': dis_opt, 'frequency': n_critic},
        {'optimizer': gen_opt, 'frequency': 1}
    )

> 需要知道的一些事情：
>* Lightning 根据需要在每个优化器和学习率调度器上调用 .backward() 和 .step() 。
>* 如果您使用 16 位精度（精度=16），Lightning 将自动处理优化器。
>* 如果您使用多个优化器，则 training_step() 将有一个额外的 optimizer_idx 参数。
>* 如果您使用 torch.optim.LBFGS，Lightning 会自动为您处理关闭功能。
>* 如果使用多个优化器，则在每个训练步骤仅针对当前优化器的参数计算梯度。
>* 如果您需要控制这些优化器执行或覆盖默认 .step() 计划的频率，请覆盖 optimizer_step() 挂钩。

### foward
```
LightningModule.forward(*args, **kwargs)
```

与 `torch.nn.Module.forward()` 相同。

参数
* `*args` – 无论你决定传递给 `forward` 方法。
* `**kwargs` – 关键字参数也是可能的。

返回类型  
任何

返回  
你的模型输出

### freeze
```
LightningModule.freeze()
```

冻结所有参数以进行推理。

例子：

In [ ]:
model = MyLightningModule(...)
model.freeze()

### log
```
LightningModule.log(name, value, prog_bar=False, logger=True, on_step=None, on_epoch=None, reduce_fx='default', tbptt_reduce_fx=None, tbptt_pad_token=None, enable_graph=False, sync_dist=False, sync_dist_op=None, sync_dist_group=None, add_dataloader_idx=True, batch_size=None, metric_attribute=None, rank_zero_only=None)
```

记录一个键值对。

例子：

In [ ]:
self.log('train_loss', loss)

每个钩子的默认行为如下：

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtwxujqipkj617p0u075w02.jpg)

**参数：**

name – 登录键
* value – 要记录的值。 可以是浮点数、张量、度量或前者的字典。
* prog_bar – 如果 True 记录到进度条
* logger – 如果 True 记录到记录器
* on_step – 如果 True 在此步骤记录。 在training_step 没有自动记录但没有validation/test_step
* on_epoch – 如果 True 记录纪元累积指标。 没有在 val/test 步骤自动记录，但不是 training_step
* reduce_fx – Epoch结束时步长值的减少函数。 默认情况下，torch.mean()。
* enable_graph – 如果为 True，则不会自动分离图形
* sync_dist – 如果为 True，则减少跨 GPU/TPU 的指标
* sync_dist_group – 要同步的 ddp 组
* add_dataloader_idx – 如果为 True，则将当前数据加载器的索引附加到名称（使用多个时）。 如果为 False，则用户需要为每个数据加载器提供唯一名称，以免混合值
* batch_size – 当前批次大小。 这将直接从加载的批次中推断出来，但某些数据结构可能需要显式提供它。
* metric_attribute – 为了恢复度量状态，Lightning 需要模型中的 torchmetrics.Metric 的引用。 如果它是模型属性，则会自动找到它。
* rank_zero_only – 该值是否仅记录在 rank 0 上。这将防止同步，因为并非所有进程都会执行此日志调用，因此会产生死锁。

### log_dict
```
LightningModule.log_dict(dictionary, prog_bar=False, logger=True, on_step=None, on_epoch=None, reduce_fx='default', tbptt_reduce_fx=None, tbptt_pad_token=None, enable_graph=False, sync_dist=False, sync_dist_op=None, sync_dist_group=None, add_dataloader_idx=True, batch_size=None, rank_zero_only=None)
```

一次记录一个值字典。

例子：

In [ ]:
values = {'loss': loss, 'acc': acc, ..., 'metric_n': metric_n}
self.log_dict(values)

参数:
* dictionary (Mapping[str, Union[Metric, Tensor, Number, Mapping[str, Union[Metric, Tensor, Number]]]]) – 键值对。 值可以是浮点数、张量、度量或前者的字典。
* prog_bar (bool) – 如果 True 记录到进度库
* logger (bool) – 如果 True 记录到记录器
* on_step (Optional[bool]) – 如果 True 在此步骤记录。 training_step 没有自动记录，但没有验证/test_step
* on_epoch (Optional[bool]) – 如果 True 记录纪元累积指标。 没有自动记录 val/test 步骤但不是 training_step
* reduce_fx (Union[str, Callable]) – 纪元结束时步长值的减少函数。 默认情况下，torch.mean()。
* enable_graph (bool) – 如果为 True，则不会自动分离图形
* sync_dist (bool) – 如果为 True，则减少跨 GPU/TPU 的指标
* sync_dist_group (Optional[Any]) – ddp 组同步
* add_dataloader_idx (bool) – 如果为 True，则将当前数据加载器的索引附加到名称（使用多个时）。 如果为 False，则用户需要为每个数据加载器提供唯一名称，以免混合值
* batch_size (Optional[int]) – 当前批次大小。 这将直接从加载的批次中推断出来，但某些数据结构可能需要显式提供它。
* rank_zero_only (Optional[bool]) – 该值是否仅记录在 rank 0 上。这将防止同步，因为并非所有进程都会执行此日志调用，因此会产生死锁。

### manual_backward
```
LightningModule.manual_backward(loss, *args, **kwargs)
```

手动进行优化时，直接从您的 `training_step()` 调用它。 通过使用它，Lightning 可以确保在使用混合精度时应用所有正确的缩放比例。

有关更多示例，请参阅手动优化。

例子：

In [ ]:
def training_step(...):
    opt = self.optimizers()
    loss = ...
    opt.zero_grad()
    # 自动应用缩放等...
    self.manual_backward(loss)
    opt.step()

参数
* `loss (Tensor)` – 计算梯度的张量。 必须附上图表。
* `*args` – 要转发到`backward（）`的其他位置参数
* `**kwargs` – 要转发到 `backward()` 的其他关键字参数

### print
```
LightningModule.print(*args, **kwargs)
```

仅从进程 0 打印。在任何分布式模式下使用它只记录一次。

参数
* `*args` – 要打印的东西。 与 Python 的内置打印功能相同。
* `**kwargs` – 与 Python 的内置打印功能相同。

例子：

In [ ]:
def forward(self, x):
    self.print(x, 'in forward')

### predict_step
```
LightningModule.predict_step(batch, batch_idx, dataloader_idx=None)
```

在 `predict()` 期间调用的步进函数。 默认情况下，它调用 `forward()`。 覆盖以添加任何处理逻辑。

`predict_step()` 用于在多设备上扩展推理。

为了防止 OOM 错误，可以使用 BasePredictionWriter 回调在每批之后或在 epoch 结束时将预测写入磁盘或数据库。

BasePredictionWriter 应该在使用基于 spawn 的加速器时使用。 这种情况发生在 `Trainer(accelerator="ddp_spawn")` 或使用 `Trainer(tpu_cores=8)` 在 8 个 TPU 核心上训练，因为不会返回预测。

例子

In [ ]:
class MyModel(LightningModule):

    def predicts_step(self, batch, batch_idx, dataloader_idx):
        return self(batch)

dm = ...
model = MyModel()
trainer = Trainer(gpus=2)
predictions = trainer.predict(model, dm)

参数
* batch（任何）– 当前批次
* batch_idx (int) – 当前批次的索引
* dataloader_idx (Optional[int]) – 当前数据加载器的索引

返回  
预测输出

### save_hyperparameters
```
LightningModule.save_hyperparameters(*args, ignore=None, frame=None, logger=True)
```

将参数保存到 hparams 属性。

参数
* args – dict、NameSpace 或 OmegaConf 的单个对象或来自 __init__ 类的字符串名称或参数
* ignore (Union[Sequence[str], str, None]) – 要忽略的类 __init__ 中的参数名称或参数名称列表
* frame (Optional[frame]) – 一个框架对象。 默认为无
* logger (bool) – 是否将超参数发送到记录器。 默认值：真

例子：

In [ ]:
class ManuallyArgsModel(HyperparametersMixin):
    def __init__(self, arg1, arg2, arg3):
        super().__init__()
        # 手动分配参数
        self.save_hyperparameters('arg1', 'arg3')
    def forward(self, *args, **kwargs):
        ...
model = ManuallyArgsModel(1, 'abc', 3.14)
model.hparams
"arg1": 1
"arg3": 3.14

In [ ]:
class AutomaticArgsModel(HyperparametersMixin):
    def __init__(self, arg1, arg2, arg3):
        super().__init__()
        # 等效自动
        self.save_hyperparameters()
    def forward(self, *args, **kwargs):
        ...
model = AutomaticArgsModel(1, 'abc', 3.14)
model.hparams
"arg1": 1
"arg2": abc
"arg3": 3.14

In [ ]:
class SingleArgModel(HyperparametersMixin):
    def __init__(self, params):
        super().__init__()
        # 手动分配单个参数
        self.save_hyperparameters(params)
    def forward(self, *args, **kwargs):
        ...
model = SingleArgModel(Namespace(p1=1, p2='abc', p3=3.14))
model.hparams
"p1": 1
"p2": abc
"p3": 3.14

In [ ]:
class ManuallyArgsModel(HyperparametersMixin):
    def __init__(self, arg1, arg2, arg3):
        super().__init__()
        # 将要忽略的参数作为字符串或在列表中传递
        self.save_hyperparameters(ignore='arg2')
    def forward(self, *args, **kwargs):
        ...
model = ManuallyArgsModel(1, 'abc', 3.14)
model.hparams
"arg1": 1
"arg3": 3.14

### test_step
```
LightningModule.test_step(*args, **kwargs)
```

对来自测试集的单批数据进行操作。 在此步骤中，您通常会生成示例或计算任何感兴趣的内容，例如准确性。

In [ ]:
# 这些调用的伪代码
test_outs = []
for test_batch in test_data:
    out = test_step(test_batch)
    test_outs.append(out)
test_epoch_end(test_outs)

参数
* batch (Tensor | (Tensor, …) | [Tensor, …]) – DataLoader 的输出。 张量、元组或列表。
* batch_idx (int) – 该批次的索引。
* dataloader_idx (int) – 生成此批次的数据加载器的索引（仅当使用多个测试数据加载器时）。

返回类型
联合[张量，字典[str，任何]，无]

返回  
任何。
* 任何对象或值
* 无 - 测试将跳到下一批

In [ ]:
# 如果您有一个测试数据加载器：
def test_step(self, batch, batch_idx):
    ...


# 如果您有多个测试数据加载器：
def test_step(self, batch, batch_idx, dataloader_idx):
    ...

例子：

In [ ]:
# CASE 1: 单个测试数据集
def test_step(self, batch, batch_idx):
    x, y = batch

    # 实现你自己的
    out = self(x)
    loss = self.loss(out, y)

    # 记录 6 个示例图像
    # 或生成的文本...或其他
    sample_imgs = x[:6]
    grid = torchvision.utils.make_grid(sample_imgs)
    self.logger.experiment.add_image('example_images', grid, 0)

    # 计算acc
    labels_hat = torch.argmax(out, dim=1)
    test_acc = torch.sum(y == labels_hat).item() / (len(y) * 1.0)

    # 记录输出！
    self.log_dict({'test_loss': loss, 'test_acc': test_acc})

如果您传入多个测试数据加载器，则 test_step() 将有一个额外的参数。

In [ ]:
# CASE 2: 多个测试数据加载器
def test_step(self, batch, batch_idx, dataloader_idx):
    # dataloader_idx 告诉您这是哪个数据集。
    ...

> 如果不需要测试，则不需要实现此方法。

> 当 test_step() 被调用时，模型已被置于 eval 模式并且 PyTorch 梯度已被禁用。 在测试阶段结束时，模型返回训练模式并启用梯度。

### test_step_end
```
LightningModule.test_step_end(*args, **kwargs)
```

在使用 dp 或 ddp2 进行测试时使用此选项，因为 test_step() 将仅对批处理的一部分进行操作。 但是，这仍然是可选的，并且仅在诸如 softmax 或 NCE 损失之类的情况下才需要。

> 如果您稍后切换到 ddp 或其他某种模式，它仍会被调用，因此您不必更改代码。

In [ ]:
# pseudocode
sub_batches = split_batches_for_dp(batch)
batch_parts_outputs = [test_step(sub_batch) for sub_batch in sub_batches]
test_step_end(batch_parts_outputs)

参数
* `batch_parts_outputs` – 您在 test_step() 中为每个批处理部分返回的内容。

返回类型  
Union[Tensor, Dict[str, Any], None]

In [ ]:
# 没有 test_step_end
# 如果在DP或者DDP2中使用，这个batch是1/num_gpus大
def test_step(self, batch, batch_idx):
    # 批次为 1/num_gpus 大
    x, y = batch

    out = self(x)
    loss = self.softmax(out)
    self.log("test_loss", loss)


# --------------
# 使用 test_step_end 对整批进行 softmax
def test_step(self, batch, batch_idx):
    # 批次为 1/num_gpus 大
    x, y = batch

    out = self.encoder(x)
    return out


def test_step_end(self, output_results):
    # 这现在是批次的全尺寸
    all_test_step_outs = output_results.out
    loss = nce_loss(all_test_step_outs)
    self.log("test_loss", loss)

### test_epoch_end
```
LightningModule.test_epoch_end(outputs)
```

在测试Epoch结束时调用所有测试步骤的输出。

In [ ]:
# 这些调用的伪代码
test_outs = []
for test_batch in test_data:
    out = test_step(test_batch)
    test_outs.append(out)
test_epoch_end(test_outs)

参数  
输出 (List[Union[Tensor, Dict[str, Any]]]) – 您在 test_step_end() 中定义的输出列表，或者如果有多个数据加载器，则包含每个数据加载器的输出列表的列表

> 如果您没有定义 test_step()，则不会调用它。

例子

使用单个数据加载器：

In [ ]:
def test_epoch_end(self, outputs):
    # 对所有测试批次的输出做一些事情
    all_test_preds = test_step_outputs.predictions

    some_result = calc_all_results(all_test_preds)
    self.log(some_result)

对于多个数据加载器，输出将是一个列表列表。 外部列表包含每个数据加载器的一个条目，而内部列表包含该数据加载器的每个测试步骤的单独输出。

In [ ]:
def test_epoch_end(self, outputs):
    final_value = 0
    for dataloader_outputs in outputs:
        for test_step_out in dataloader_outputs:
            # do something
            final_value += test_step_out

    self.log("final_metric", final_value)

### to_onnx
```
LightningModule.to_onnx(file_path, input_sample=None, **kwargs)
```

以 ONNX 格式保存模型。

参数
* `file_path (Union[str, Path])` – onnx 模型应保存到的文件路径。
* `input_sample (Optional[Any])` – 用于跟踪的输入。 默认值：无（使用 self.example_input_array）
* `**kwargs – 将传递给 torch.onnx.export` 函数。

例子

In [ ]:
class SimpleModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = torch.nn.Linear(in_features=64, out_features=4)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

In [ ]:
with tempfile.NamedTemporaryFile(suffix='.onnx', delete=False) as tmpfile:
    model = SimpleModel()
    input_sample = torch.randn((1, 64))
    model.to_onnx(tmpfile.name, input_sample, export_params=True)
    os.path.isfile(tmpfile.name)

True

### to_torchscript
```
LightningModule.to_torchscript(file_path=None, method='script', example_inputs=None, **kwargs)
```

默认情况下将整个模型编译为 ScriptModule。 如果要使用跟踪，请提供参数 method='trace' 并确保提供了 example_inputs 参数，或者模型设置了 example_input_array。 如果您想自定义脚本化模块，您应该覆盖此方法。 如果您想返回多个模块，我们建议使用字典。

参数
* `file_path (Union[str, Path, None])` – 保存火炬脚本的路径。 默认值：无（不保存文件）。
* `method (Optional[str])` – 是否使用 TorchScript 的脚本或跟踪方法。 默认值：‘脚本’
* `example_inputs (Optional[Any])` – 当方法设置为“trace”时用于进行跟踪的输入。 默认值：无（使用 `example_input_array`）
* `**kwargs` – 将传递给 `torch.jit.script()` 或 `torch.jit.trace()`` 函数的附加参数。

> * 需要实现 forward() 方法。  
> * 导出的脚本将设置为评估模式。  
> * 建议您安装最新支持的 PyTorch 版本以不受限制地使用此功能。 另请参阅 torch.jit 文档了解支持的功能。

例子

In [ ]:
class SimpleModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = torch.nn.Linear(in_features=64, out_features=4)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

model = SimpleModel()
torch.jit.save(model.to_torchscript(), "model.pt")  
os.path.isfile("model.pt")  
torch.jit.save(model.to_torchscript(file_path="model_trace.pt", method='trace', 
                                    example_inputs=torch.randn(1, 64)))  
os.path.isfile("model_trace.pt")  

True

返回类型  
Union[ScriptModule, Dict[str, ScriptModule]]

返回  
无论 `file_path` 是否定义，此 LightningModule 作为 torchscript。

### training_step
```
LightningModule.training_step(*args, **kwargs)
```

在这里你计算并返回训练损失和一些额外的指标，例如 进度条或记录器。

参数
* `batch (Tensor | (Tensor, …) | [Tensor, …])` – DataLoader 的输出。 张量、元组或列表。
* `batch_idx (int)` – 该批次的整数显示索引
* `optimizer_idx (int)` – 当使用多个优化器时，这个参数也会出现。
* `hiddens (Tensor)` – 如果 `truncated_bptt_steps > 0`，则传入。

返回类型  
Union[Tensor, Dict[str, Any]]

返回
任何。
* Tensor - 损失张量
* dict - 字典。 可以包含任何键，但必须包含键“loss”
* 无 - 训练将跳到下一批

> 多 GPU 或 TPU 或启用 16 位精度当前不支持返回 None。

在这一步中，您通常会进行前向传递并计算批次的损失。 你还可以做更有趣的事情，比如多次向前传球或特定于模型的东西。

例子：

In [ ]:
def training_step(self, batch, batch_idx):
    x, y, z = batch
    out = self.encoder(x)
    loss = self.loss(out, x)
    return loss

如果您定义了多个优化器，则将使用附加的 optimizer_idx 参数调用此步骤。

In [ ]:
# Multiple optimizers (e.g.: GANs)
def training_step(self, batch, batch_idx, optimizer_idx):
    if optimizer_idx == 0:
        # do training_step with encoder
        ...
    if optimizer_idx == 1:
        # do training_step with decoder
        ...

如果您通过时间添加截断的反向传播，您还将获得带有上一步隐藏状态的附加参数。

In [ ]:
# Truncated back-propagation through time
def training_step(self, batch, batch_idx, hiddens):
    # hiddens are the hidden states from the previous truncated backprop step
    ...
    out, hiddens = self.lstm(data, hiddens)
    ...
    return {"loss": loss, "hiddens": hiddens}

> 进度条中显示的损失值在最后一个值上进行了平滑（平均），因此它不同于训练/验证步骤中返回的实际损失。

### training_step_end
```
LightningModule.training_step_end(*args, **kwargs)
```

在使用 dp 或 ddp2 进行训练时使用此选项，因为 training_step() 将仅对批次的一部分进行操作。 但是，这仍然是可选的，并且仅在诸如 softmax 或 NCE 损失之类的情况下才需要。

> 如果您稍后切换到 ddp 或其他一些模式，它仍然会被调用，这样您就不必更改您的代码

In [ ]:
# pseudocode
sub_batches = split_batches_for_dp(batch)
batch_parts_outputs = [training_step(sub_batch) for sub_batch in sub_batches]
training_step_end(batch_parts_outputs)

参数  
batch_parts_outputs – 您在 training_step 中为每个批次部分返回的内容。

当使用 dp/ddp2 分布式后端时，只有一部分批次在 training_step 中：

In [ ]:
def training_step(self, batch, batch_idx):
    # batch is 1/num_gpus big
    x, y = batch

    out = self(x)

    # softmax 仅使用分母中批次的一部分
    loss = self.softmax(out)
    loss = nce_loss(loss)
    return loss

如果你想对批次的所有部分做一些事情，那么使用这个方法来做：

In [ ]:
def training_step(self, batch, batch_idx):
    # batch is 1/num_gpus big
    x, y = batch

    out = self.encoder(x)
    return {"pred": out}


def training_step_end(self, training_step_outputs):
    gpu_0_pred = training_step_outputs[0]["pred"]
    gpu_1_pred = training_step_outputs[1]["pred"]
    gpu_n_pred = training_step_outputs[n]["pred"]

    # 这个 softmax 现在使用完整的批处理
    loss = nce_loss([gpu_0_pred, gpu_1_pred, gpu_n_pred])
    return loss

### training_epoch_end
```
LightningModule.training_epoch_end(outputs)
```

在训练阶段结束时使用所有训练步骤的输出调用。 如果您需要对 `training_step()` 返回的所有输出执行某些操作，请使用此选项。

In [ ]:
# the pseudocode for these calls
train_outs = []
for train_batch in train_data:
    out = training_step(train_batch)
    train_outs.append(out)
training_epoch_end(train_outs)

参数  
输出 (List[Union[Tensor, Dict[str, Any]]]) – 您在 training_step() 中定义的输出列表，或者如果有多个数据加载器，则包含每个数据加载器的输出列表的列表。

返回类型  
None

返回  
None

> 如果此方法未被覆盖，则不会调用此方法。

例子：

In [ ]:
def training_epoch_end(self, training_step_outputs):
    # 对所有 training_step 输出做一些事情
    return result

对于多个数据加载器，输出将是一个列表列表。 外部列表包含每个数据加载器的一个条目，而内部列表包含该数据加载器的每个训练步骤的各个输出。

In [ ]:
def training_epoch_end(self, training_step_outputs):
    for out in training_step_outputs:
        ...

### unfreeze
```
LightningModule.unfreeze()
```

解冻所有训练参数。

In [ ]:
model = MyLightningModule(...)
model.unfreeze()

### validation_step
```
LightningModule.validation_step(*args, **kwargs)
```

对验证集中的单批数据进行操作。 在此步骤中，您可能会生成示例或计算任何感兴趣的内容，例如准确性。

In [ ]:
# the pseudocode for these calls
val_outs = []
for val_batch in val_data:
    out = validation_step(val_batch)
    val_outs.append(out)
validation_epoch_end(val_outs)

参数
* `batch (Tensor | (Tensor, …) | [Tensor, …])` – DataLoader 的输出。 张量、元组或列表。
* `batch_idx (int)` – 该批次的索引
* `dataloader_idx (int)` – 生成此批次的数据加载器的索引（仅当使用多个 val 数据加载器时）

返回类型  
Union[Tensor, Dict[str, Any], None]

返回  
任何对象或值
无 - 验证将跳到下一批

In [ ]:
# pseudocode of order
val_outs = []
for val_batch in val_data:
    out = validation_step(val_batch)
    if defined("validation_step_end"):
        out = validation_step_end(out)
    val_outs.append(out)
val_outs = validation_epoch_end(val_outs)

In [ ]:
# if you have one val dataloader:
def validation_step(self, batch, batch_idx):
    ...


# if you have multiple val dataloaders:
def validation_step(self, batch, batch_idx, dataloader_idx):
    ...

例子：

In [ ]:
# CASE 1: 单个验证数据集
def validation_step(self, batch, batch_idx):
    x, y = batch

    # implement your own
    out = self(x)
    loss = self.loss(out, y)

    # log 6 example images
    # or generated text... or whatever
    sample_imgs = x[:6]
    grid = torchvision.utils.make_grid(sample_imgs)
    self.logger.experiment.add_image('example_images', grid, 0)

    # calculate acc
    labels_hat = torch.argmax(out, dim=1)
    val_acc = torch.sum(y == labels_hat).item() / (len(y) * 1.0)

    # log the outputs!
    self.log_dict({'val_loss': loss, 'val_acc': val_acc})

如果传入多个 val 数据加载器，validation_step() 将有一个额外的参数。

In [ ]:
# CASE 2: 多个验证数据加载器
def validation_step(self, batch, batch_idx, dataloader_idx):
    # dataloader_idx 告诉您这是哪个数据集。
    ...

> 如果不需要验证，则不需要实现此方法。

> 当调用validation_step() 时，模型已被置于评估模式并且PyTorch 梯度已被禁用。 在验证结束时，模型返回训练模式并启用梯度。

### validation_step_end
```
LightningModule.validation_step_end(*args, **kwargs)
```

在使用 dp 或 ddp2 进行验证时使用此选项，因为 validation_step() 将仅对批处理的一部分进行操作。 但是，这仍然是可选的，并且仅在诸如 softmax 或 NCE 损失之类的情况下才需要。

如果您稍后切换到 ddp 或其他某种模式，它仍会被调用，因此您不必更改代码。

In [ ]:
# pseudocode
sub_batches = split_batches_for_dp(batch)
batch_parts_outputs = [validation_step(sub_batch) for sub_batch in sub_batches]
validation_step_end(batch_parts_outputs)

参数  
batch_parts_outputs – 您在 validation_step() 中为每个批处理部分返回的内容。

返回类型    
Union[Tensor, Dict[str, Any], None]

返回  
没有或任何东西

In [ ]:
# 没有validation_step_end
# 如果在DP或者DDP2中使用，这个batch是1/num_gpus大
def validation_step(self, batch, batch_idx):
    # batch is 1/num_gpus big
    x, y = batch

    out = self.encoder(x)
    loss = self.softmax(out)
    loss = nce_loss(loss)
    self.log("val_loss", loss)


# --------------
# 使用validation_step_end 对整批进行softmax
def validation_step(self, batch, batch_idx):
    # batch is 1/num_gpus big
    x, y = batch

    out = self(x)
    return out


def validation_step_end(self, val_step_outputs):
    for out in val_step_outputs:
        ...

#### validation_epoch_end
```
LightningModule.validation_epoch_end(outputs)
```

在验证时期结束时使用所有验证步骤的输出调用。

In [ ]:
# the pseudocode for these calls
val_outs = []
for val_batch in val_data:
    out = validation_step(val_batch)
    val_outs.append(out)
validation_epoch_end(val_outs)

参数  
输出 (List[Union[Tensor, Dict[str, Any]]]) – 您在 validation_step() 中定义的输出列表，或者如果有多个数据加载器，则包含每个数据加载器的输出列表的列表。

返回类型  
没有任何

返回。
没有任何

> 如果你没有定义一个validation_step()，它不会被调用。

例子

使用单个数据加载器：

In [ ]:
def validation_epoch_end(self, val_step_outputs):
    for out in val_step_outputs:
        ...

对于多个数据加载器，输出将是一个列表列表。 外部列表包含每个数据加载器的一个条目，而内部列表包含该数据加载器的每个验证步骤的各个输出。

In [ ]:
def validation_epoch_end(self, outputs):
    for dataloader_output_result in outputs:
        dataloader_outs = dataloader_output_result.dataloader_i_outputs

    self.log("final_metric", final_value)

## 属性

这些是 LightningModule 中可用的属性。

### current_epoch

现在的epoch

In [ ]:
def training_step(self):
    if self.current_epoch == 0:
        ...

### device

模块所在的设备。 使用它来保持您的代码设备不可知

In [ ]:
def training_step(self):
    z = torch.rand(2, 3, device=self.device)

### global_rank

此 LightningModule 的 global_rank。 Lightning 仅从 global_rank = 0 保存日志、权重等。您通常不需要使用此属性

全局排名是指该 GPU 在所有 GPU 中的索引。 例如，如果使用 10 台机器，每台机器有 4 个 GPU，则第 10 台机器上的第 4 个 GPU 的 global_rank = 39

### global_step

当前步骤（不重置每个 epoch）

In [ ]:
def training_step(self):
    self.logger.experiment.log_image(..., step=self.global_step)

### hparams

通过 `__init__()` 传递的调用 `save_hyperparameters` 保存的参数
可以通过 hparams 属性访问。

In [ ]:
def __init__(self, learning_rate):
    self.save_hyperparameters()


def configure_optimizers(self):
    return Adam(self.parameters(), lr=self.hparams.learning_rate)

### logger

当前正在使用的记录器（张量板或其他支持的记录器）

In [ ]:
def training_step(self):
    # 通用记录器（无论是张量板还是其他支持的记录器都相同）
    self.logger

    # 特定的记录器
    tensorboard_logger = self.logger.experiment

### local_rank

此 LightningModule 的 local_rank。 Lightning 仅从 global_rank = 0 保存日志、权重等。您通常不需要使用此属性

本地排名是指该机器上的排名。 例如，如果使用 10 台机器，则每台机器上索引 0 处的 GPU 的 local_rank = 0。

### precision

使用的精度类型：

In [ ]:
def training_step(self):
    if self.precision == 16:
        ...

### trainer

指向训练器的指针

In [ ]:
def training_step(self):
    max_steps = self.trainer.max_steps
    any_flag = self.trainer.any_flag

### use_amp

如果使用自动混合精度 (AMP)，则为真

### automatic_optimization

设置为 False 时，Lightning 不会自动执行优化过程。 这意味着您负责处理优化器。 但是，我们确实会注意精度和使用的任何加速器。

有关详细信息，请参阅手动优化。

In [ ]:
def __init__(self):
    self.automatic_optimization = False


def training_step(self, batch, batch_idx):
    opt = self.optimizers(use_pl_optimizer=True)

    loss = ...
    opt.zero_grad()
    self.manual_backward(loss)
    opt.step()

仅当使用 2+ 优化器并且您知道如何正确执行优化过程时才建议这样做。 请注意，通过依赖 `optimizer_idx` 参数，自动优化仍然可以与多个优化器一起使用。 手动优化对于强化学习、稀疏编码和 GAN 研究等研究主题最有用。

In [ ]:
def __init__(self):
    self.automatic_optimization = False


def training_step(self, batch, batch_idx):
    # 使用 use_pl_optimizer=False 访问您的优化器。 默认为真
    opt_a, opt_b = self.optimizers(use_pl_optimizer=True)

    gen_loss = ...
    opt_a.zero_grad()
    self.manual_backward(gen_loss)
    opt_a.step()

    disc_loss = ...
    opt_b.zero_grad()
    self.manual_backward(disc_loss)
    opt_b.step()

### example_input_array

设置和访问 example_input_array ，它基本上是一个批处理。

In [ ]:
def __init__(self):
    self.example_input_array = ...
    self.generator = ...


def on_train_epoch_end(self):
    # 使用 example_input_array 生成一些图像
    gen_images = self.generator(self.example_input_array)

### datamodule

设置或访问您的数据模块。

In [ ]:
def configure_optimizers(self):
    num_training_samples = len(self.trainer.datamodule.train_dataloader())
    ...

### model_size

使用 LightningModule 中的 `self.model_size` 获取模型文件大小（以兆字节为单位）。

## truncated_bptt_steps

截断的反向传播间隔在更长的序列中每 k 步执行一次反向传播。 这是通过将沿时间维度分割成大小为 k 的分割的训练批次传递给 training_step 来实现的。 为了保持相同的前向传播行为，所有隐藏状态都应该保持在每个时间维度分割之间。

如果启用此选项，您的批次将自动被截断，并且训练器将对其应用截断的反向传播。

（威廉姆斯等人。“一种有效的基于梯度的算法，用于循环网络轨迹的在线训练。”）

In [ ]:
from pytorch_lightning import LightningModule


class MyModel(LightningModule):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        # batch_first 必须设置为 True
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
        )

        ...

        # 重要提示：此属性激活随时间截断的反向传播
        # 将此值设置为 2 会将批次拆分为大小为 2 的序列
        self.truncated_bptt_steps = 2

    # 通过时间截断反向传播
    def training_step(self, batch, batch_idx, hiddens):
        x, y = batch

        # 必须更新训练步骤以接受 ``hiddens`` 参数
        # hiddens 是前一个被截断的反向传播步骤的隐藏
        out, hiddens = self.lstm(x, hiddens)

        ...

        return {"loss": ..., "hiddens": hiddens}

Lightning 负责沿时间维度拆分您的批次。 它被假定为批次的第二个维度。 因此，在上面的示例中，我们设置了 batch_first=True。

In [ ]:
# 我们使用第二个作为时间维度
# (batch, time, ...)
sub_batch = batch[0, 0:t, ...]

要修改批处理的拆分方式，请重载 pytorch_lightning.core.LightningModule.tbptt_split_batch()：

In [ ]:
class LitMNIST(LightningModule):
    def tbptt_split_batch(self, batch, split_size):
        # 对批次进行自己的拆分
        return splits

## Hooks

这是描述 fit() 结构的伪代码。 为简单起见，未表示每个函数的输入和输出。 请查看每个函数的 API 参考以获取更多信息。

In [ ]:
def fit(self):
    if global_rank == 0:
        # prepare data is called on GLOBAL_ZERO only
        prepare_data()

    configure_callbacks()

    with parallel(devices):
        # devices can be GPUs, TPUs, ...
        train_on_device(model)


def train_on_device(model):
    # called PER DEVICE
    on_fit_start()
    setup("fit")
    configure_optimizers()

    on_pretrain_routine_start()
    on_pretrain_routine_end()

    # the sanity check runs here

    on_train_start()
    for epoch in epochs:
        train_loop()
    on_train_end()

    on_fit_end()
    teardown("fit")


def train_loop():
    on_epoch_start()
    on_train_epoch_start()

    for batch in train_dataloader():
        on_train_batch_start()

        on_before_batch_transfer()
        transfer_batch_to_device()
        on_after_batch_transfer()

        training_step()

        on_before_zero_grad()
        optimizer_zero_grad()

        on_before_backward()
        backward()
        on_after_backward()

        on_before_optimizer_step()
        optimizer_step()

        on_train_batch_end()

        if should_check_val:
            val_loop()
    # end training epoch
    training_epoch_end()

    on_train_epoch_end()
    on_epoch_end()


def val_loop():
    on_validation_model_eval()  # calls `model.eval()`
    torch.set_grad_enabled(False)

    on_validation_start()
    on_epoch_start()
    on_validation_epoch_start()

    for batch in val_dataloader():
        on_validation_batch_start()

        on_before_batch_transfer()
        transfer_batch_to_device()
        on_after_batch_transfer()

        validation_step()

        on_validation_batch_end()
    validation_epoch_end()

    on_validation_epoch_end()
    on_epoch_end()
    on_validation_end()

    # set up for train
    on_validation_model_train()  # calls `model.train()`
    torch.set_grad_enabled(True)

### backward
```
LightningModule.backward(loss, optimizer, optimizer_idx, *args, **kwargs)
```

调用以对 training_step() 中返回的损失执行向后执行。 如果需要，请使用您自己的实现覆盖此挂钩。

参数
* loss (Tensor) – training_step() 返回的损失张量。 如果使用梯度累积，这里的损失保持归一化值（按 1 / 累积步骤缩放）。
* optimizer (Optional[Optimizer]) – 当前使用的优化器。 如果使用手动优化，则无。
* optimizer_idx (Optional[int]) – 当前使用的优化器的索引。 如果使用手动优化，则无。

例子：

In [ ]:
def backward(self, loss, optimizer, optimizer_idx):
    loss.backward()

### get_progress_bar_dict
```
LightningModule.get_progress_bar_dict()
```

实现它以覆盖进度条中显示的默认项目。 默认情况下，它包括平均损失值、BPTT 的拆分索引（如果使用）和使用记录器时的实验版本。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtx22wwgn8j6180044aa902.jpg)

以下是如何覆盖默认值的示例：

In [ ]:
def get_progress_bar_dict(self):
    # 不显示版本号
    items = super().get_progress_bar_dict()
    items.pop("v_num", None)
    return items

返回类型  
Dict[str, Union[int, str]]

返回  
包含要显示在进度条中的项目的字典。

### on_before_backward
```
ModelHooks.on_before_backward(loss)
```

在 `loss.backward()` 之前调用。

参数  
`loss`（张量）——损失除以梯度积累的批次数，如果使用原生 AMP，则按比例缩放。

### on_after_backward
```
ModelHooks.on_after_backward()
```

在 `loss.backward()` 之后和优化器被执行之前调用。

> 如果使用原生 AMP，此时梯度不会被取消缩放。 如果您需要未缩放的梯度，请使用 on_before_optimizer_step。

### on_before_zero_grad
```
ModelHooks.on_before_zero_grad(optimizer)
```

在 training_step() 之后和 optimizer.zero_grad() 之前调用。

在执行优化器步骤之后和归零梯度之前在训练循环中调用。 检查重量信息并更新重量的好地方。

这就是它被调用：

In [ ]:
for optimizer in optimizers:
    out = training_step(...)

    model.on_before_zero_grad(optimizer) # < ---- 在这里调用
    optimizer.zero_grad()

    backward()

参数  
optimizer（optimizer）– 应该将其归零的优化器。

返回类型  
None

### on_fit_start
```
ModelHooks.on_fit_start()
```

在 fit 开始时调用。 如果在 DDP 上，它会在每个进程上调用

### on_fit_end
```
ModelHooks.on_fit_end()
```

在拟合结束时调用。 如果在 DDP 上，它会在每个进程上调用

### on_load_checkpoint
```
CheckpointHooks.on_load_checkpoint（检查点）
```

由 Lightning 调用以恢复您的模型。 如果你用 on_save_checkpoint() 保存了一些东西，这是你恢复它的机会。

参数  
checkpoint (Dict[str, Any]) – 加载的检查点

例子：

In [ ]:
def on_load_checkpoint(self, checkpoint):
    # 99% 的情况下你不需要实现这个方法
    self.something_cool_i_want_to_save = checkpoint['something_cool_i_want_to_save']

Lightning 自动恢复全局步长、Epoch和训练状态，包括放大器缩放。 您无需恢复任何有关培训的内容。

### on_save_checkpoint
```
CheckpointHooks.on_save_checkpoint(checkpoint)
```

保存检查点时由闪电调用，让您有机会存储您可能想要保存的任何其他内容。

参数  
checkpoint (Dict[str, Any]) – 转储到文件之前的完整检查点字典。 这个钩子的实现可以在这个字典中插入额外的数据。

例子：

In [ ]:
def on_save_checkpoint(self, checkpoint):
    # 99% 的用例你不需要实现这个方法
    checkpoint['something_cool_i_want_to_save'] = my_cool_pickable_object

> Lightning 保存了训练的所有方面（epoch、全局步骤等），包括放大器缩放。 您无需存储任何有关培训的信息。

### on_train_start
```
ModelHooks.on_train_start()
```

在健全性检查后在训练开始时调用。

### on_train_end
```
ModelHooks.on_train_end()
```

在记录器实验关闭之前在训练结束时调用。

### on_validation_start
```
ModelHooks.on_validation_start()
```

在验证开始时调用。

### on_validation_end
```
ModelHooks.on_validation_end()
```
在验证结束时调用。

### on_pretrain_routine_start
```
ModelHooks.on_pretrain_routine_start()[来源]
```
在预训练例程开始时调用（在拟合和训练开始之间）。
* fit
* pretrain_routine start
* pretrain_routine end
* training_start

### on_pretrain_routine_end
```
ModelHooks.on_pretrain_routine_end()[来源]
```
在训练前例程结束时调用（在拟合和训练开始之间）。
* fit
* pretrain_routine start
* pretrain_routine end
* training_start

### on_test_batch_start
```
ModelHooks.on_test_batch_start(batch, batch_idx, dataloader_idx)
```

在该批次发生任何事情之前在测试循环中调用。

参数
* batch (Any) – 测试 DataLoader 返回的批处理数据。
* batch_idx (int) – 批次索引
* dataloader_idx (int) – 数据加载器的索引

### on_test_batch_end
```
ModelHooks.on_test_batch_end（输出，批次，batch_idx，dataloader_idx)
```
批处理后在测试循环中调用。

参数
* outputs (Union[Tensor, Dict[str, Any], None]) – test_step_end(test_step(x)) 的输出
* batch (Any) – 测试 DataLoader 返回的批处理数据。
* batch_idx (int) – 批次索引
* dataloader_idx (int) – 数据加载器的索引

### on_test_epoch_start
```
ModelHooks.on_test_epoch_start()
```
在Epoch开始时在测试循环中调用。

### on_test_epoch_end
```
ModelHooks.on_test_epoch_end()
```
在 epoch 结束时在测试循环中调用。

### on_test_start
```
ModelHooks.on_test_start()
```
在测试开始时调用。

### on_test_end
```
ModelHooks.on_test_end()
```
在测试结束时调用。

### on_train_batch_start
```
ModelHooks.on_train_batch_start(batch, batch_idx, dataloader_idx)
```

在该批次发生任何事情之前在训练循环中调用。

如果您在此处返回 -1，您将跳过当前 epoch 剩余时间的训练。

参数
* batch (Any) – 训练 DataLoader 返回的批处理数据。
* batch_idx (int) – 批次的索引
* dataloader_idx (int) – 数据加载器的索引

### on_train_batch_end
```
ModelHooks.on_train_batch_end(outputs, batch, batch_idx, dataloader_idx)
```

在批处理之后在训练循环中调用。

参数
* outputs (Union[Tensor, Dict[str, Any]]) – training_step_end(training_step(x)) 的输出
* batch (Any) – 训练 DataLoader 返回的批处理数据。
* batch_idx (int) – 批次的索引
* dataloader_idx (int) – 数据加载器的索引

### on_epoch_start
```
ModelHooks.on_epoch_start()
```
在任一训练/验证/测试时期开始时调用。

### on_epoch_end
```
ModelHooks.on_epoch_end()
```
当任一训练/验证/测试时期结束时调用。

### on_train_epoch_start
```
ModelHooks.on_train_epoch_start()
```
在 epoch 开始时在训练循环中调用。

### on_train_epoch_end
```
ModelHooks.on_train_epoch_end()
```
在 epoch 结束时在训练循环中调用。

要在 epoch 结束时访问所有批处理输出，请执行以下任一操作：
1. 在 LightningModule 中实现 training_epoch_end 或
2. 在 LightningModule 的属性上跨步骤缓存数据并在此挂钩中访问它们

### on_validation_batch_start
```
ModelHooks.on_validation_batch_start(batch, batch_idx, dataloader_idx)
```
在该批次发生任何事情之前在验证循环中调用。

参数
* batch (Any) – 验证 DataLoader 返回的批处理数据。
* batch_idx (int) – 批次的索引
* dataloader_idx (int) – 数据加载器的索引

### on_validation_batch_end
```
ModelHooks.on_validation_batch_end（输出，批次，batch_idx，dataloader_idx）
```

在批处理之后在验证循环中调用。

参数
* outputs (Union[Tensor, Dict[str, Any], None]) – validation_step_end(validation_step(x)) 的输出
* batch (Any) – 验证 DataLoader 返回的批处理数据。
* batch_idx (int) – 批次的索引
* dataloader_idx (int) – 数据加载器的索引

### on_validation_epoch_start
```
ModelHooks.on_validation_epoch_start()
```
在纪元开始时在验证循环中调用。

### on_validation_epoch_end
```
ModelHooks.on_validation_epoch_end()
```
在纪元结束时在验证循环中调用。

### on_post_move_to_device
```
ModelHooks.on_post_move_to_device()
```
在调用 to() 之后在 parameter_validation 装饰器中调用。 这是将模块移动到设备后在模块之间绑定权重的好地方。 可在 TPU 上训练具有权重共享属性的模型时使用。

解决了 TPU 上共享权重的处理：https://github.com/pytorch/xla/blob/master/TROUBLESHOOTING.md#xla-tensor-quirks

例子：

In [ ]:
def on_post_move_to_device(self):
    self.decoder.weight = self.encoder.weight

### on_validation_model_eval
```
ModelHooks.on_validation_model_eval()
```
在 val 循环期间将模型设置为 eval

### on_validation_model_train
```
ModelHooks.on_validation_model_train()
```
将模型设置为在 val 循环期间进行训练

### on_test_model_eval
```
ModelHooks.on_test_model_eval()
```
在测试循环期间将模型设置为 eval

### on_test_model_train
```
ModelHooks.on_test_model_train()
```
将模型设置为在测试循环期间进行训练

### on_before_optimizer_step
```
ModelHooks.on_before_optimizer_step(optimizer, optimizer_idx)
```
在 optimizer.step() 之前调用。

仅当不需要累积梯度时才调用钩子。 请参阅：accumulate_grad_batches。 如果使用本机 AMP，则在调用此钩子之前将不缩放损失。 有关渐变缩放的更多信息，请参阅这些文档。

参数
* optimizer(Optimizer) – 当前使用的优化器。
* optimizer_idx (int) – 当前使用的优化器的索引。
例子：

In [ ]:
def on_before_optimizer_step(self, optimizer, optimizer_idx):
    # 检查张量板中的梯度信息的示例
    if self.trainer.global_step % 25 == 0:  # don't make the tf file huge
        for k, v in self.named_parameters():
            self.logger.experiment.add_histogram(
                tag=k, values=v.grad, global_step=self.trainer.global_step
            )

### optimizer_step
```
LightningModule.optimizer_step(epoch=None, batch_idx=None, optimizer=None, optimizer_idx=None, optimizer_closure=None, on_tpu=None, using_native_amp=None, using_lbfgs=None)[source]
```

覆盖此方法以调整 Trainer 调用每个优化器的默认方式。 默认情况下，Lightning 会调用 step() 和 zero_grad()，如示例中所示，每个优化器都会调用一次。 当 Trainer(accumulate_grad_batches != 1) 时，在累积阶段不会调用此方法（和 zero_grad()）。

> 如果您要覆盖此方法，请确保将 optimizer_closure 参数传递给 optimizer.step() 函数，如示例中所示。 这确保了 training_step()、optimizer.zero_grad()、backward() 在训练循环中被调用。

参数
* epoch (Optional[int]) – Current epoch
* batch_idx (Optional[int]) – Index of current batch
* optimizer (Optional[Optimizer]) – A PyTorch optimizer
* optimizer_idx (Optional[int]) – 如果您使用了多个优化器，则此索引到该列表中。
* optimizer_closure (Optional[Callable]) – Closure for all optimizers
* on_tpu (Optional[bool]) – True if TPU backward is required
* using_native_amp (Optional[bool]) – True if using native amp
* using_lbfgs (Optional[bool]) – True if the matching optimizer is torch.optim.LBFGS

例子：

In [ ]:
# DEFAULT
def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx,
                   optimizer_closure, on_tpu, using_native_amp, using_lbfgs):
    optimizer.step(closure=optimizer_closure)

# 优化器步骤的交替计划（即：GAN）
def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx,
                   optimizer_closure, on_tpu, using_native_amp, using_lbfgs):
    # 更新生成器选择每一步
    if optimizer_idx == 0:
        optimizer.step(closure=optimizer_closure)

    # 每 2 步更新鉴别器 opt
    if optimizer_idx == 1:
        if (batch_idx + 1) % 2 == 0 :
            optimizer.step(closure=optimizer_closure)

    # ...
    # add as many optimizers as you want

这是另一个示例，展示了如何将其用于更高级的事情，例如学习率热身：

In [ ]:
# learning rate warm-up
def optimizer_step(
    self,
    epoch,
    batch_idx,
    optimizer,
    optimizer_idx,
    optimizer_closure,
    on_tpu,
    using_native_amp,
    using_lbfgs,
):
    # warm up lr
    if self.trainer.global_step < 500:
        lr_scale = min(1.0, float(self.trainer.global_step + 1) / 500.0)
        for pg in optimizer.param_groups:
            pg["lr"] = lr_scale * self.learning_rate

    # update params
    optimizer.step(closure=optimizer_closure)

### optimizer_zero_grad
```
LightningModule.optimizer_zero_grad(epoch, batch_idx, optimizer, optimizer_idx)
```

覆盖此方法以更改 optimizer.zero_grad() 的默认行为。

参数
* epoch (int) – 当前纪元
* batch_idx (int) – 当前批次的索引
* optimizer（Optimizer）——一个 PyTorch 优化器
* optimizer_idx (int) – 如果您使用了多个优化器，则该索引到该列表中。
例子：

In [ ]:
# DEFAULT
def optimizer_zero_grad(self, epoch, batch_idx, optimizer, optimizer_idx):
    optimizer.zero_grad()

# 将梯度设置为“None”而不是零以提高性能。
def optimizer_zero_grad(self, epoch, batch_idx, optimizer, optimizer_idx):
    optimizer.zero_grad(set_to_none=True)

> 有关上述示例的说明，请参阅 torch.optim.Optimizer.zero_grad()。

### prepare_data
```
LightningModule.prepare_data()
```

使用它来下载和准备数据。

> 不要为模型设置状态（改为使用设置），因为这不会在 DDP/TPU 中的每个 GPU 上调用

例子：

In [ ]:
def prepare_data(self):
    # good
    download_data()
    tokenize()
    etc()

    # bad
    self.split = data_split
    self.some_state = some_other_state()

在 DDP 中可以通过两种方式调用 prepare_data（使用 Trainer(prepare_data_per_node)）：

1. 每个节点一次。 这是默认设置，仅在 LOCAL_RANK=0 时调用。
2. 总共一次。 仅在 GLOBAL_RANK=0 时调用。
例子：

In [ ]:
# DEFAULT
# 每个节点在该节点的 LOCAL_RANK=0 上调用一次
Trainer(prepare_data_per_node=True)

# 调用 GLOBAL_RANK=0（非常适合共享文件系统）
Trainer(prepare_data_per_node=False)

> 不推荐使用 trainer 标志设置 prepare_data_per_node 并将在 v1.7.0 中删除。 请直接在 LightningDataModule 或 LightningModule 中设置 prepare_data_per_node。

这是在请求数据加载器之前调用的：

In [ ]:
model.prepare_data()
initialize_distributed()
model.setup(stage)
model.train_dataloader()
model.val_dataloader()
model.test_dataloader()

### setup
```
DataHooks.setup(stage=None)
```

在拟合开始时调用（训练 + 验证）、验证、测试和预测。 当您需要动态构建模型或调整模型时，这是一个很好的钩子。 使用 DDP 时，每个进程都会调用此钩子。

参数  
stage (Optional[str]) – either 'fit', 'validate', 'test', or 'predict'

例子：

In [ ]:
class LitModel(...):
    def __init__(self):
        self.l1 = None

    def prepare_data(self):
        download_data()
        tokenize()

        # don't do this
        self.something = else

    def setup(stage):
        data = Load_data(...)
        self.l1 = nn.Linear(28, data.num_classes)

### tbptt_split_batch
```
LightningModule.tbptt_split_batch(batch, split_size)
```

在使用截断的时间反向传播时，必须沿时间维度拆分每个批次。 默认情况下，Lightning 会处理此问题，但对于自定义行为，请覆盖此功能。

参数
* batch (Tensor) – Current batch
* split_size (int) – The size of the split

返回类型  
列表

返回
批量拆分列表。 每个拆分将传递给 training_step() 以启用随时间截断的反向传播。 默认实现在dim=1（即时间dim）处拆分根级张量和序列。 它假设每次dim 的长度都相同。

例子：

In [ ]:
def tbptt_split_batch(self, batch, split_size):
  splits = []
  for t in range(0, time_dims[0], split_size):
      batch_split = []
      for i, x in enumerate(batch):
          if isinstance(x, torch.Tensor):
              split_x = x[:, t:t + split_size]
          elif isinstance(x, collections.Sequence):
              split_x = [None] * len(x)
              for batch_idx in range(len(x)):
                  split_x[batch_idx] = x[batch_idx][t:t + split_size]

          batch_split.append(split_x)

      splits.append(batch_split)

  return splits

> 如果 truncated_bptt_steps > 0，则在 on_batch_start() 之后在训练循环中调用。每个返回的批次拆分单独传递给 training_step()。

### teardown
```
DataHooks.teardown(stage=None)
```

在拟合结束时调用（训练 + 验证）、验证、测试、预测或调整。

参数  
stage (Optional[str]) – 'fit'、'validate'、'test' 或 'predict'

返回类型
无

### train_dataloader
```
DataHooks.train_dataloader()
```

实施一个或多个 PyTorch DataLoader 进行训练。

返回类型  
`Union[DataLoader, Sequence[DataLoader], Sequence[Sequence[DataLoader]], Sequence[Dict[str, DataLoader]], Dict[str, DataLoader], Dict[str, Dict[str, DataLoader]], Dict[str, Sequence[DataLoader]]]`

返回  
指定训练样本的 torch.utils.data.DataLoader 的集合。 如果有多个数据加载器，请参阅此页面。

除非您将 reload_dataloaders_every_n_epochs 设置为正整数，否则您返回的数据加载器将不会重新加载。

对于数据处理，请使用以下模式：
* 在 prepare_data() 中下载
* 在 setup() 中处理和拆分

但是，以上仅是分布式处理所必需的。

> 不要在 prepare_data 中分配状态

* fit()
* …
* prepare_data()
* setup()
* train_dataloader()

> Lightning 为分布式和任意硬件添加了正确的采样器。 无需自己设置。

例子：

In [ ]:
# single dataloader
def train_dataloader(self):
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.5,), (1.0,))])
    dataset = MNIST(root='/path/to/mnist/', train=True, transform=transform,
                    download=True)
    loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=self.batch_size,
        shuffle=True
    )
    return loader

# multiple dataloaders, return as list
def train_dataloader(self):
    mnist = MNIST(...)
    cifar = CIFAR(...)
    mnist_loader = torch.utils.data.DataLoader(
        dataset=mnist, batch_size=self.batch_size, shuffle=True
    )
    cifar_loader = torch.utils.data.DataLoader(
        dataset=cifar, batch_size=self.batch_size, shuffle=True
    )
    # each batch will be a list of tensors: [batch_mnist, batch_cifar]
    return [mnist_loader, cifar_loader]

# multiple dataloader, return as dict
def train_dataloader(self):
    mnist = MNIST(...)
    cifar = CIFAR(...)
    mnist_loader = torch.utils.data.DataLoader(
        dataset=mnist, batch_size=self.batch_size, shuffle=True
    )
    cifar_loader = torch.utils.data.DataLoader(
        dataset=cifar, batch_size=self.batch_size, shuffle=True
    )
    # each batch will be a dict of tensors: {'mnist': batch_mnist, 'cifar': batch_cifar}
    return {'mnist': mnist_loader, 'cifar': cifar_loader}

### val_dataloader
```
DataHooks.val_dataloader()
```

实现一个或多个 PyTorch DataLoader 以进行验证。

除非您将 reload_dataloaders_every_n_epochs 设置为正整数，否则您返回的数据加载器将不会重新加载。

建议所有数据下载和准备都在 prepare_data() 中进行。

* fit()
* …
* prepare_data()
* train_dataloader()
* val_dataloader()
* test_dataloader()

> Lightning为分布式和任意硬件添加了正确的采样器，无需自己设置。

返回类型  
Union[DataLoader, Sequence[DataLoader]]

返回：  
A torch.utils.data.DataLoader or 它们的序列指定验证样本。

例子：

In [ ]:
def val_dataloader(self):
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.5,), (1.0,))])
    dataset = MNIST(root='/path/to/mnist/', train=False,
                    transform=transform, download=True)
    loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=self.batch_size,
        shuffle=False
    )

    return loader

# can also return multiple dataloaders
def val_dataloader(self):
    return [loader_a, loader_b, ..., loader_n]

> 如果不需要验证数据集和validation_step()，则不需要实现此方法。

> 在您返回多个验证数据加载器的情况下，validation_step() 将有一个参数 dataloader_idx 与此处的顺序匹配。

### test_dataloader
```
DataHooks.test_dataloader()
```

实现一个或多个 PyTorch DataLoader 进行测试。

除非您将 `reload_dataloaders_every_n_epochs` 设置为正整数，否则您返回的数据加载器不会重新加载。

对于数据处理，请使用以下模式：
* 在 `prepare_data()` 中下载
* 在 `setup()` 中处理和拆分

但是，以上仅是分布式处理所必需的。

> 不要在 prepare_data 中分配状态

* fit()
* …
* prepare_data()
* setup()
* train_dataloader()
* val_dataloader()
* test_dataloader()

> Lightning 为分布式和任意硬件添加了正确的采样器。 无需自己设置。

返回类型  
联合[DataLoader, Sequence[DataLoader]]

返回  
torch.utils.data.DataLoader 或它们的序列指定测试样本。

例子：

In [ ]:
def test_dataloader(self):
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.5,), (1.0,))])
    dataset = MNIST(root='/path/to/mnist/', train=False, transform=transform,
                    download=True)
    loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=self.batch_size,
        shuffle=False
    )

    return loader

# can also return multiple dataloaders
def test_dataloader(self):
    return [loader_a, loader_b, ..., loader_n]

> 如果不需要测试数据集和 test_step()，则不需要实现此方法。

> 如果您返回多个测试数据加载器，则 test_step() 将有一个参数 dataloader_idx 与此处的顺序匹配。

### transfer_batch_to_device
```
DataHooks.transfer_batch_to_device(batch, device, dataloader_idx)
```

如果您的 DataLoader 返回包装在自定义数据结构中的张量，则覆盖此挂钩。

下面列出的数据类型（以及它们的任意嵌套）是开箱即用的：
* torch.Tensor 或任何实现 .to(...)
* 列表
* 字典
* 元组
* torchtext.data.batch.Batch

对于其他任何事情，您需要定义如何将数据移动到目标设备（CPU、GPU、TPU 等）。

> 这个钩子应该只传输数据而不修改它，也不应该将数据移动到任何其他设备而不是作为参数传入的设备（除非你知道你在做什么）。 要检查此钩子的当前执行状态，您可以使用 self.trainer.training/testing/validating/predicting 以便您可以根据需要添加不同的逻辑。

> 该钩子仅在单 GPU 训练和 DDP（无数据并行）上运行。 数据并行支持将在不久的将来出现。

参数
* batch (Any) – 需要传输到新设备的一批数据。
* device (device) – PyTorch 中定义的目标设备。
* dataloader_idx (int) – 批处理所属的数据加载器的索引。

返回类型  
任何

返回。
对新设备上数据的引用。

例子：

In [ ]:
def transfer_batch_to_device(self, batch, device):
    if isinstance(batch, CustomBatch):
        # 将自定义数据结构中的所有张量移动到设备
        batch.samples = batch.samples.to(device)
        batch.targets = batch.targets.to(device)
    else:
        batch = super().transfer_batch_to_device(data, device)
    return batch

### on_before_batch_transfer
```
DataHooks.on_before_batch_transfer(batch, dataloader_idx)
```

覆盖以在批次传输到设备之前更改或应用批次扩充。

> 要检查此钩子的当前执行状态，您可以使用 self.trainer.training/testing/validating/predicting 以便您可以根据需要添加不同的逻辑。

> 该钩子仅在单 GPU 训练和 DDP（无数据并行）上运行。 数据并行支持将在不久的将来出现。

参数
* batch (Any) – 需要更改或扩充的一批数据。
* dataloader_idx (int) – 批处理所属的数据加载器的索引。
返回类型 
任何

返回  
一批数据

例子：

In [ ]:
def on_before_batch_transfer(self, batch, dataloader_idx):
    batch['x'] = transforms(batch['x'])
    return batch

### on_after_batch_transfer
```
DataHooks.on_after_batch_transfer(batch, dataloader_idx)
```

在将批次传输到设备后，覆盖以更改或应用批次扩充到您的批次。

> 检查此钩子的当前执行状态，您可以使用 self.trainer.training/testing/validating/predicting 以便您可以根据需要添加不同的逻辑。

> 该钩子仅在单 GPU 训练和 DDP（无数据并行）上运行。 数据并行支持将在不久的将来出现。

参数
* batch (Any) – 需要更改或扩充的一批数据。
* dataloader_idx (int) – 批处理所属的数据加载器的索引。
返回类型  
任何

返回  
一批数据

例子：

In [ ]:
def on_after_batch_transfer(self, batch, dataloader_idx):
    batch['x'] = gpu_transforms(batch['x'])
    return batch

### add_to_queue
```
LightningModule.add_to_queue(queue)
```

将 trainer.callback_metrics 字典附加到给定的队列。 为了避免内存共享问题，我们将数据转换为 numpy。

参数  
queue (SimpleQueue) – 要追加数据的队列实例。

### get_from_queue
```
LightningModule.get_from_queue(queue)
```

从给定队列中检索 trainer.callback_metrics 字典。 为了保持一致性，我们将数据回传到 torch.Tensor。

参数  
queue (SimpleQueue) – 从哪里获取数据的队列实例。